In [1]:
import rebound
import reboundx
import sys
sys.path.append('../systemsounds')
import systemsounds as ss
import numpy as np
from random import random, uniform, seed
from systemsounds import EventRecorder
import pandas as pd

In [2]:
def acrit(sim): # Holman & Wiegert 99 for P type circumbinary
    ps = sim.particles
    mu = ps[1].m/(ps[1].m+ps[0].m)
    return 1.60 + 5.10*ps[1].e -2.22*ps[1].e**2 + 4.12*mu -4.27*ps[1].e*mu -5.09*mu**2 + 4.61*ps[1].e**2*mu**2 

def stop(reb_sim, reb_col):
    reb_sim.contents._status = 6
    return 0

def distance(sim, i1, i2):
    ps = sim.particles
    diff = ps[i2]-ps[i1]
    return np.sqrt(diff.x**2 + diff.y**2 + diff.z**2)

class mindRecorder(EventRecorder):
    def __init__(self, sim, starindex):
        self.starindex = starindex
        self.mind = 0.99*min(distance(sim, 0, 2), distance(sim, 1,2)) # needs to start less than initial value
        self.mind0 = self.mind
        super(mindRecorder, self).__init__(sim, lambda sim, i: self.mind-distance(sim, 2, self.starindex), targets=[None])
    def process_event(self, event_sim, target=None):
        self.mind = distance(event_sim, 2, self.starindex)
        super(mindRecorder, self).process_event(event_sim, target)

In [3]:
def runbinary(randseed=0, Rmin=1.e-2, mratio=None, ebinary=None):
    print(randseed)
    seed(randseed)
    if mratio is None:
        mratio = random()
    if ebinary is None:
        ebinary = 0.9*random()
        
    sim = rebound.Simulation()
    sim.G = 4*np.pi**2
    sim.add(m=1., r=Rmin)
    sim.add(m=mratio, a=1., e=ebinary, r=Rmin, pomega=random()*2.*np.pi, f=random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    sim.add(a=1.2*acrit(sim), f=random()*2.*np.pi)
    
    sim.exit_max_distance= 100.
    sim.collision="direct"
    sim.collision_resolve = stop

    rebx = reboundx.Extras(sim)
    rebx.add("modify_orbits_forces")
    ps[2].params['tau_a'] = -1000*ps[2].P
    
    recorder1 = mindRecorder(sim, 0)
    recorder2 = mindRecorder(sim, 1)
    tmax=1.e4
    
    code=0
    try:
        sim.integrate(tmax)
    except KeyboardInterrupt:
        code=1
    except rebound.Escape:
        code=2
    
    with open('Nbody{0}.txt'.format(Rmin), 'a') as f:
        f.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\n'.format(randseed, recorder1.mind, recorder2.mind, recorder1.mind0, mratio, ebinary, code))

In [4]:
def RunRmin(Rmin, Rminprev):
    seeds, Nbodymind1s, Nbodymind2s, mind0s, mratios, ebinarys, codes = np.loadtxt('Nbody{0}.txt'.format(Rminprev)).T
    seeds = np.array([int(seed) for seed in seeds])
    mask = (codes == 1)
    print(len(seeds[mask]))
    print(seeds[mask])
    for i in seeds[mask]:
        runbinary(i, Rmin)

In [ ]:
RunRmin(1.e-3, 1.e-2)

59
[  67  125  215  251  269  309  316  353  399  422  473  474  489  522  524
  547  552  557  651  678  737  849  944  946  954 1029 1051 1055 1067 1127
 1212 1332 1345 1358 1374 1385 1427 1434 1479 1481 1521 1530 1600 1605 1716
 1742 1751 1757 1759 1768 1824 1839 1847 1902 1916 1918 1925 1944 1980]
67
125
215


In [5]:
RunRmin(1.e-6, 1.e-3)

59
[  67  125  215  251  269  309  316  353  399  422  473  474  489  522  524
  547  552  557  651  678  737  849  944  946  954 1029 1051 1055 1067 1127
 1212 1332 1345 1358 1374 1385 1427 1434 1479 1481 1521 1530 1600 1605 1716
 1742 1751 1757 1759 1768 1824 1839 1847 1902 1916 1918 1925 1944 1980]
67
125
215
251
269
309
316
353
399
422
473
474
489
522
524
547
552
557
651
678
737
849
944
946
954
1029
1051
1055
1067
1127
1212
1332
1345
1358
1374
1385
1427
1434
1479
1481
1521
1530
1600
1605
1716
1742
1751
1757
1759
1768
1824
1839
1847
1902
1916
1918
1925
1944
1980


In [6]:
RunRmin(1.e-8, 1.e-6)

3
[ 737 1605 1916]
737
1605
1916
